## Podstawy Kerasa

### Sieć neuronowa z dwoma warstwami ukrytymi

In [1]:
# Uciszenie ostrzeżeń z TensorFlow
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from keras import layers, models

model = models.Sequential()

# Warstwa definiująca kształt danych wejściowych
# Zazwyczaj jest to pierwsza warstwa modelu
model.add(layers.Input(shape=(28, 28, 1)))

# Warstwa zmieniająca kształt danych wejściowych
# (bs, 28, 28, 1) -> (bs, 28 * 28 * 1)
model.add(layers.Reshape([28 * 28 * 1]))

# Warstwa w pełni połączona
# Odpowiednikiem warstwy Linear w PyTorch jest warstwa Dense w Keras
model.add(layers.Dense(128, activation="relu", use_bias=False))

# Aktwyację można definiować również jako osobną warstwę
model.add(layers.Dense(256, use_bias=False))
model.add(layers.Activation("relu"))

# Warstwa Dropout
model.add(layers.Dense(128))
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.5))

# Warstwa wyjściowa
# W przypadku klasyfikacji wieloklasowej używamy funkcji softmax
model.add(layers.Dense(10, activation="softmax"))


# Wyświetlenie podsumowania modelu
model.summary()


E0000 00:00:1733353573.998822 1000309 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733353574.001825 1000309 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1733353575.747598 1000309 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21769 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:06:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,306 (653.54 KB)

 Trainable params: 167,306 (653.54 KB)

 Non-trainable params: 0 (0.00 B)

### Trenowanie sieci na zbiorze danych MNIST

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Keras posiada wbudowane zbiory danych
from keras import layers, models, datasets

model = models.Sequential()
model.add(layers.Input(shape=(28, 28, 1)))
model.add(layers.Reshape([28 * 28 * 1]))
model.add(layers.Dense(128, activation="relu", use_bias=False))
model.add(layers.Dense(256, use_bias=False))
model.add(layers.Activation("relu"))
model.add(layers.Dense(128))
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation="softmax"))


# Wczytanie zbioru danych MNIST
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalizacja danych
train_images = train_images / 255.0
test_images = test_images / 255.0

print(train_images.shape, train_labels.shape)

# Kompilacja modelu
# W przypadku klasyfikacji wieloklasowej używamy sparse_categorical_crossentropy
# Funkcja ta działa podobnie do categorical_crossentropy, ale nie wymaga one-hot encoding
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Trenowanie modelu
# 10 epok, batch size = 32, walidacja przy pomocy 20% danych treningowych
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Ewaluacja modelu
loss, accuracy = model.evaluate(test_images, test_labels)

print(f"Loss: {loss}, Accuracy: {accuracy}")


(60000, 28, 28) (60000,)
Epoch 1/10


I0000 00:00:1733353577.707927 1000399 service.cc:148] XLA service 0x7eff080065a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733353577.707959 1000399 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
I0000 00:00:1733353577.783428 1000399 cuda_dnn.cc:529] Loaded cuDNN version 90300


  76/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4354 - loss: 1.6857

I0000 00:00:1733353579.877608 1000399 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8340 - loss: 0.5344 - val_accuracy: 0.9602 - val_loss: 0.1313
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9605 - loss: 0.1361 - val_accuracy: 0.9655 - val_loss: 0.1145
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9727 - loss: 0.0932 - val_accuracy: 0.9669 - val_loss: 0.1169
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9786 - loss: 0.0734 - val_accuracy: 0.9722 - val_loss: 0.1030
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9830 - loss: 0.0567 - val_accuracy: 0.9701 - val_loss: 0.1157
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9842 - loss: 0.0513 - val_accuracy: 0.9734 - val_loss: 0.1055
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9867 - loss: 0.0435 - val_accuracy: 0.9708 - val_loss: 0.1230
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9881 - loss: 0.0390 - val_accurac

### Łączenie wielu sieci sekwencyjnych w jedną sieć

In [7]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Keras posiada wbudowane zbiory danych
from keras import layers, models

model = models.Sequential()
model.add(layers.Input(shape=(28, 28, 1)))

blok_conv = models.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Reshape([-1]),  # flatten
    ],
    name="blok_conv",
)

blok_dense = models.Sequential(
    [
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dense(256),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dropout(0.5),
    ],
    name="blok_dense",
)

model.add(blok_conv)
model.add(blok_dense)
model.add(layers.Dense(10, activation="softmax", name="output"))

model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ blok_conv (Sequential)          │ (None, 5408)           │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ blok_dense (Sequential)         │ (None, 128)            │       760,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 761,930 (2.91 MB)

 Trainable params: 760,906 (2.90 MB)

 Non-trainable params: 1,024 (4.00 KB)